In [121]:

#procurar no buscape
#verificar se estão no range
#Salvar as ofertas boas num Dataframe
#exportar para excel
#enviar por email

In [134]:
#Criar um navegador
#Observação: utilizar Edge ou Chrome
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.microsoft import EdgeChromiumDriverManager 

from selenium.webdriver.edge.service import Service
servico = Service(EdgeChromiumDriverManager().install())
navegador = webdriver.Edge(service=servico)

#importar base de dados
import pandas as pd
tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)


,Nome,Termos Banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 2060,galax,1800,2500


Definição de função do Google Shopping e Buscapé

In [135]:
def busca_google_shopping(navegador,produto, termos_banidos, preco_minimo, preco_maximo):
    #tratar valores
    produto = produto.lower() #garantir que os dados da tabela fiquem minusculos
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")#transforma a string de termos banidos em lista
    lista_termos_nome_produto = produto.split(" ")#transforma a string de produtos em lista
    lista_ofertas = [] # lista vazia para armazenar os resultados da busca
    #transformar os dados do excel em float
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    #entrar no google
    navegador.get('https://www.google.com/')
    
    #busca o elemento pelo xpath e envia o produto
    navegador.find_element('xpath', '//*[@id="APjFqb"]').send_keys(produto)
    navegador.find_element('xpath', '//*[@id="APjFqb"]').send_keys(Keys.ENTER)
    #entrar no google shopping com condicional
    elementos2 = navegador.find_elements('class name', 'hdtb-mitem') #versao mais antiga
    elementos = navegador.find_elements('class name', 'GKS7s') #versao mais nova
    for item in elementos:
        if "Shopping" in item.text:
            print("v1 - "+item.text)
            item.click()
            break
        
    for item2 in elementos2:
        if "Shopping" in item2.text:
            print("v2 - "+item2.text)
            item2.click()
            break 
            
           
    
    #Pegar preços dos produtos google shopping 
    lista_resultados = navegador.find_elements('class name', 'i0X6df')
    
    #a busca ocorre apenas dentro da classe i0X6df e naõ em no navegador inteiro
    #esse elemento é o card do produto
    
    for resultado in lista_resultados:
        
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()
        #analisar se o nome não tem nenhum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
                
        #analisar se tem todos os termos buscados no nome do produto
        tem_termos_nome_produto = True
        for palavra in lista_termos_nome_produto:
            if palavra not in nome:
                tem_termos_nome_produto = False
        
        #Tratamento preco
        if not tem_termos_banidos and tem_termos_nome_produto:
            preco = resultado.find_element('class name', 'a8Pemb').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".").replace("+impostos", " ")
            #remover o R$ ; remover o espaço; remover o ponto; trocar a virgula por ponto para reconhecer no float
            preco = float(preco)
            
            
            
            
            #Verificar se o preço está entre o preço minimo e maximo
            if preco_minimo <= preco <= preco_maximo:
                #Caso retornar None é necessário utilizar uma referencia
                elemento_referencia = resultado.find_element('class name', 'bONr3b') #Elemento filho da classe do Link 
                elemento_pai = elemento_referencia.find_element('xpath', ('..'))#Elemento pai do elemento filho (..) serve para voltar um nível
                link = elemento_pai.get_attribute('href') #retornando None em vez do link
                lista_ofertas.append((nome,preco,link))

    return lista_ofertas

def busca_buscape(navegador,produto, termos_banidos, preco_minimo, preco_maximo):
    #trata valores
    produto = produto.lower() #garantir que os dados da tabela fiquem minusculos
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")#transforma a string de termos banidos em lista
    lista_termos_nome_produto = produto.split(" ")#transforma a string de produtos em lista
    lista_ofertas = [] # lista vazia para armazenar os resultados da busca
    preco_minimo = float(preco_minimo)#converte dados do excel em float
    preco_maximo = float(preco_maximo)
    #entrar no buscape
    navegador.get("https://www.buscape.com.br/")
    
    #buscar o elemento pelo xpath e envia o produto
    navegador.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto)
    navegador.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.ENTER)
    
    #pegar os resultados
    
    #analisar se o resultado tem termo banidos e todos termos do produto
    
    #analisar range de preço determidado
    
    #retorna lista de ofertas buscape

In [136]:
#Dados do produto
produto  = "iphone 12 64 gb"
termos_banidos = "mini watch"
preco_minimo = 3000
preco_maximo = 3500

busca_buscape(navegador,produto, termos_banidos, preco_minimo, preco_maximo)


In [128]:
lista_ofertas_google_shopping = busca_google_shopping(navegador,produto, termos_banidos, preco_minimo, preco_maximo)

v2 - Shopping


In [129]:
print(lista_ofertas_google_shopping)

[('iphone 12 apple 64gb tela de 6.1 polegadas câmera 12mp ios azul', 3390.64, 'https://www.google.com/url?url=https://www.lebiscuit.com.br/iphone-12-apple-64gb-tela-de-6-1-polegadas-camera-12mp-ios-16597al08l832478/p%3Fidsku%3D2147376583&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwi_mo-Vk8-CAxX8LbkGHflfCxEQgOUECPAN&usg=AOvVaw3aXmVzkM-BEQx3lzco5ZxI'), ('iphone 12 64gb azul - excelente', 3299.0, 'https://www.google.com/url?url=https://www.trocafone.com.br/iphone-12-64gb-azul---excelente-70613/p%3Fidsku%3D39869&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwi_mo-Vk8-CAxX8LbkGHflfCxEQguUECMAP&usg=AOvVaw34E-2JoB1C5D5hWcN8oo8c'), ('iphone 12 64gb preto, super conservado! | celular iphone | usado', 3454.0, 'https://www.google.com/url?url=https://www.enjoei.com.br/p/iphone-12-64gb-preto-super-conservado-92855609%3Fg_campaign%3Dgoogle_shopping%26srsltid%3DAfmBOooPNaPanvoQr-F_HmPJJ-mEO6oCZBbvmuGFD16cHW115H6IvYzB6Cs&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwi_mo-Vk8-CAxX8LbkGHflfCxEQgOUECO8P&us